In [1]:
import pandas as pd
import requests
import json
from pprint import pprint
from config import api_key
from census import Census
from us import states

ModuleNotFoundError: No module named 'config'

In [ ]:
# Setting up lists for looping and data retrieval
start_date = [
    "2020-12-31T00:00:00Z",
    "2021-01-31T00:00:00Z",
    "2021-02-28T00:00:00Z",
    "2021-03-31T00:00:00Z",
    "2021-04-30T00:00:00Z",
    "2021-05-31T00:00:00Z",
    "2021-06-30T00:00:00Z",
    "2021-07-31T00:00:00Z",
    "2021-08-31T00:00:00Z",
    "2021-09-30T00:00:00Z",
    "2021-10-31T00:00:00Z",
    "2021-11-30T00:00:00Z"
]

end_date = [
    "2021-02-01T00:00:00Z",
    "2021-03-01T00:00:00Z",
    "2021-04-01T00:00:00Z",
    "2021-05-01T00:00:00Z",
    "2021-06-01T00:00:00Z",
    "2021-07-01T00:00:00Z",
    "2021-08-01T00:00:00Z",
    "2021-09-01T00:00:00Z",
    "2021-10-01T00:00:00Z",
    "2021-11-01T00:00:00Z",
    "2021-12-01T00:00:00Z",
    "2022-01-01T00:00:00Z"
]

month = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec"
]

## Retrieve data from COVID19 API

Worth noting for any grader that may choose to execute the code again, it could take a while to execute the following calls to the COVID19 API. It has a tendency to time out and sometimes multiple attempts are needed. The code DOES work, it's not a problem with the code.

In [ ]:
deaths_url = f"https://api.covid19api.com/country/united-states/status/deaths?from={start_date[0]}&to={end_date[11]}&province=California"
death_data = requests.get(deaths_url).json()
death_data

In [ ]:
cases_url = f"https://api.covid19api.com/country/united-states/status/confirmed?from={start_date[0]}&to={end_date[11]}&province=California"
case_data = requests.get(cases_url).json()
case_data

## Compile retrieved data, find cumulative deaths and cases at end of each month, calculate new cases each month and compile into a dataframe

In [ ]:
# Convert Datasets to DataFrames
death_df = pd.DataFrame(death_data)
case_df = pd.DataFrame(case_data)

In [ ]:
# Creating list of unique county names
county_finder_df = death_df
clean_county_finder_df = county_finder_df[county_finder_df["City"] != "Unassigned"]
clean_county_finder_df = clean_county_finder_df[clean_county_finder_df["CityCode"] != "80006"]
county_list = clean_county_finder_df["City"].unique()
county_list.sort()
county_list

In [ ]:
# Create empty dataframe to hold all data
all_months_df = pd.DataFrame(columns=["County",
                                      "Cumulative Deaths by End of Month",
                                      "New Deaths During Month",
                                      "Cumulative Confirmed Cases by End of Month",
                                      "New Confirmed Cases During Month",
                                      "Month",
                                      "Year",
                                      "Latitude",
                                      "Longitude"]
                            )
all_months_df

In [ ]:
# Create index to tick up through all loops
index2 = 0

# Loop through all datasets
for index, date in enumerate(start_date):
    
    # Loop through dataframes
    for county in county_list:
        
        # Find cumulative deaths prior to target month, cumulative deaths at the end of the month and total deaths for the month
        county_prev_month_cum_deaths = int(death_df.loc[((death_df["City"] == county) & (death_df["Date"] == date)), "Cases"])
        county_month_cum_deaths = int(death_df.loc[((death_df["City"] == county) & (death_df["Date"] == end_date[index])), "Cases"])
        county_new_deaths = county_month_cum_deaths - county_prev_month_cum_deaths
        
        # Find cumulative cases prior to target month, cumulative cases at the end of the month and total cases for the month
        county_prev_month_cum_cases = int(case_df.loc[((case_df["City"] == county) & (case_df["Date"] == date)), "Cases"])
        county_month_cum_cases = int(case_df.loc[((case_df["City"] == county) & (case_df["Date"] == end_date[index])), "Cases"])
        county_new_cases = county_month_cum_cases - county_prev_month_cum_cases
        
        county_lat_list = death_df.loc[death_df["City"] == county, "Lat"].reset_index(drop=True)
        county_lat = county_lat_list[1]
        
        county_lon_list = death_df.loc[death_df["City"] == county, "Lon"].reset_index(drop=True)
        county_lon = county_lon_list[1]
                
        # Add values to month_df
        all_months_df.loc[index2, "County"] = county
        all_months_df.loc[index2, "Cumulative Deaths by End of Month"] = county_month_cum_deaths
        all_months_df.loc[index2, "New Deaths During Month"] = county_new_deaths
        all_months_df.loc[index2, "Cumulative Confirmed Cases by End of Month"] = county_month_cum_cases
        all_months_df.loc[index2, "New Confirmed Cases During Month"] = county_new_cases
        all_months_df.loc[index2, "Month"] = month[index]
        all_months_df.loc[index2, "Year"] = "2021"
        all_months_df.loc[index2, "Latitude"] = county_lat
        all_months_df.loc[index2, "Longitude"] = county_lon
        
        # Tick up index3
        index2 += 1
        
all_months_df

## Retrieve Census Data

In [ ]:
c = Census(api_key)
census_data = c.acs5.get(
    (
        "NAME",                   #
        "B19013_001E",            # Median Income
        "B19301_001E",            # Income per capita
        "B17001_002E",            # poverty: Number of persons whose income in the past 12 months is below the poverty level
        "B01002_001E",            # Median Age
        "B01003_001E",            # Total Population
        "B02001_002E",            # White Population
        "B02001_003E",            # Black Population
        "B02001_004E",            # American Indian population
        "B02001_005E",            # Asian population
        "B02001_008E",            # Population identifying as 2+ races
        "B03001_003E",            # Hispanic population
        "B08301_010E"             # Percent of population using public transport
    ), geo={"for": "county:*",
            "in": "state:{}".format(states.CA.fips)}
)

census_pd = pd.DataFrame(census_data)
census_pd

In [ ]:
census_pd = census_pd.rename(
    columns = {
        "NAME": "County",
        "B19013_001E": "Median Income",
        "B19301_001E": "Income per Capita",
        "B17001_002E": "Number of Poeple in Poverty",
        "B01002_001E": "Median Age",
        "B01003_001E": "Total Population",
        "B02001_002E": "Total White Population",
        "B02001_003E": "Total Black Population",
        "B02001_004E": "Total American Indian Population",
        "B02001_005E": "Total Asian Poulation",
        "B02001_008E": "Total Population Identifying as 2+ Races",
        "B03001_003E": "Total Hispanic Population",
        "B08301_010E": "Total Pop Using Public Transit"
    }
)
census_pd

In [ ]:
sorted_census_pd = census_pd.sort_values("County").reset_index(drop=True)
sorted_census_pd

In [ ]:
sorted_census_pd["County"] = county_list
sorted_census_pd.head()

In [ ]:
del sorted_census_pd["state"]
del sorted_census_pd["county"]
sorted_census_pd.head()

In [ ]:
combined_data = pd.merge(all_months_df, sorted_census_pd, on="County", how="left")
combined_data

In [ ]:
combined_data.to_csv("Resources/combined_covid_census_data.csv", index=False)

In [ ]:
#

ca_counties = len(combined_data.to_csv["County"].unique())
ca_counties